In [1]:
#!pip install yfinance
import pandas as pd
import yfinance as yf



### 1. Merging (similar to Joining)
**Definition:**
- Merging combines two DataFrames based on common columns or indices. It supports different types of joins: inner, outer, left, and right.
- Uses the `pd.merge()` function.

**Use Case in Algo Trading:**
- **Combining Historical Data**: Merge historical price data from different sources based on a common date column.
- **Integrating Indicators**: Merge technical indicators calculated on the same index or stock, using the date or timestamp as the common column.
- **Enriching Data**: Merge additional data such as fundamental data (e.g., earnings, dividends) with price data to create a more comprehensive dataset for trading algorithms.

### 2. Joining
**Definition:**
- Joining is conceptually similar to merging but typically refers to the method of combining DataFrames based on their indices using the `join()` method. This method also supports inner, outer, left, and right joins.

**Types of Joins:**
- **Inner Join**: Returns only the intersection of keys from both DataFrames.
- **Left Join**: Returns all keys from the left DataFrame and matched keys from the right DataFrame.
- **Right Join**: Returns all keys from the right DataFrame and matched keys from the left DataFrame.
- **Outer Join**: Returns all keys from both DataFrames, filling missing values with NaNs.

**Use Case in Algo Trading:**
- **Inner Join**: Useful for combining datasets where only common entries are needed. For example, combining stock prices with sentiment data, where you only care about dates where both data are available.
- **Left Join**: Useful for scenarios where you want to keep all the entries from the main dataset (e.g., stock prices) and only add supplementary data (e.g., news sentiment) if available.
- **Right Join**: Useful when the main focus is on the supplementary data, and you want to include all its entries.
- **Outer Join**: Useful for combining datasets where you want to retain all data points from both sets, filling in missing values as needed.

### 3. Concatenating
**Definition:**
- Concatenating appends DataFrames either vertically or horizontally using the `pd.concat()` function. This operation is akin to stacking DataFrames.

**Use Case in Algo Trading:**
- **Appending Historical Data**: Concatenate price data from different time periods or sources to create a continuous time series.
- **Combining Features**: Combine multiple feature DataFrames (like different technical indicators) horizontally to create a wide feature set for model training.
- **Backtesting Results**: Combine results from multiple backtesting runs or scenarios for comparison.

### Examples

#### Merging
```python
import pandas as pd

# Example DataFrames
prices = pd.DataFrame({'Date': ['2023-01-01', '2023-01-02'], 'Price': [100, 101]})
indicators = pd.DataFrame({'Date': ['2023-01-01', '2023-01-02'], 'Indicator': [0.5, 0.6]})

# Merging DataFrames
merged_data = pd.merge(prices, indicators, on='Date', how='inner')  # Inner join
print(merged_data)
```

#### Joining
```python
# Setting Date as Index
prices.set_index('Date', inplace=True)
indicators.set_index('Date', inplace=True)

# Joining DataFrames
inner_joined = prices.join(indicators, how='inner')
left_joined = prices.join(indicators, how='left')
right_joined = prices.join(indicators, how='right')
outer_joined = prices.join(indicators, how='outer')

print("Inner Join:\n", inner_joined)
print("Left Join:\n", left_joined)
print("Right Join:\n", right_joined)
print("Outer Join:\n", outer_joined)
```

#### Concatenating
```python
# Example DataFrames for Concatenation
more_prices = pd.DataFrame({'Date': ['2023-01-03', '2023-01-04'], 'Price': [102, 103]})

# Concatenating DataFrames
concatenated_data = pd.concat([prices.reset_index(), more_prices], axis=0)
print(concatenated_data)
```

### Summary
- **Merging** (or joining using `pd.merge`) allows for combining DataFrames based on common columns with different join types (inner, left, right, outer). It is useful for integrating different datasets based on specific keys.
- **Joining** (using `join()` method) also combines DataFrames based on indices and supports various join types, ideal for time series data indexed by dates.
- **Concatenating** is suitable for appending DataFrames vertically or horizontally, making it ideal for building continuous time series or combining multiple feature sets.



In [2]:
import yfinance as yf
import pandas as pd

# Define the list of tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA']

# Fetch historical price data
def fetch_historical_price_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    if 'Close' in data:
        data = data['Close'].reset_index().melt(id_vars=['Date'], var_name='ticker', value_name='close')
        data.columns = ['date', 'ticker', 'close']
    else:
        data = pd.DataFrame(columns=['date', 'ticker', 'close'])
    return data

# Fetch fundamental data (using Yahoo Finance info)
def fetch_fundamental_data(tickers):
    fundamentals = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info
        revenue = info.get('totalRevenue')
        fundamentals.append({
            'ticker': ticker,
            'quarter': 'N/A',  # Quarterly data is not directly available
            'earnings': info.get('trailingEps'),
            'revenue': revenue / 1e6 if revenue else None  # Convert to millions, handle None
        })
    return pd.DataFrame(fundamentals)

# Fetch volume data
def fetch_volume_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    if 'Volume' in data:
        data = data['Volume'].reset_index().melt(id_vars=['Date'], var_name='ticker', value_name='volume')
        data.columns = ['date', 'ticker', 'volume']
    else:
        data = pd.DataFrame(columns=['date', 'ticker', 'volume'])
    return data

# Generate synthetic trade orders (for illustration purposes)
def generate_trade_orders():
    return pd.DataFrame({
        'order_id': list(range(1, 13)),
        'date': ['2024-06-01', '2024-06-01', '2024-06-02', '2024-06-02', '2024-06-03', '2024-06-03',
                 '2024-06-04', '2024-06-04', '2024-06-05', '2024-06-05', '2024-06-06', '2024-06-06'],
        'ticker': ['AAPL', 'MSFT', 'AAPL', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META'],
        'quantity': [10, 5, 15, 7, 20, 8, 12, 10, 6, 9, 14, 7],
        'price': [150, 200, 155, 2850, 3200, 250, 700, 152, 202, 2880, 3220, 255]
    })

# Generate synthetic strategy signals (for illustration purposes)
def generate_strategy_signals():
    return pd.DataFrame({
        'date': ['2024-06-01', '2024-06-01', '2024-06-02', '2024-06-02', '2024-06-03', '2024-06-03',
                 '2024-06-04', '2024-06-04', '2024-06-05', '2024-06-05', '2024-06-06', '2024-06-06'],
        'ticker': ['AAPL', 'MSFT', 'AAPL', 'GOOGL', 'AMZN', 'META', 'TSLA', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META'],
        'signal': ['buy', 'sell', 'buy', 'sell', 'buy', 'sell', 'buy', 'sell', 'buy', 'sell', 'buy', 'sell']
    })

# Fetch data
start_date = '2024-05-01'
end_date = '2024-06-20'

historical_price_data = fetch_historical_price_data(tickers, start_date, end_date)
fundamental_data = fetch_fundamental_data(tickers)
volume_data = fetch_volume_data(tickers, start_date, end_date)
trade_orders = generate_trade_orders()
strategy_signals = generate_strategy_signals()

# Display data
print("Historical Price Data:")
print(historical_price_data.head(20))

print("\nFundamental Data:")
print(fundamental_data)

print("\nVolume Data:")
print(volume_data.head(20))

print("\nTrade Orders:")
print(trade_orders)

print("\nStrategy Signals:")
print(strategy_signals)


[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  6 of 6 completed

Historical Price Data:
         date ticker       close
0  2024-05-01   AAPL  169.300003
1  2024-05-02   AAPL  173.029999
2  2024-05-03   AAPL  183.380005
3  2024-05-06   AAPL  181.710007
4  2024-05-07   AAPL  182.399994
5  2024-05-08   AAPL  182.740005
6  2024-05-09   AAPL  184.570007
7  2024-05-10   AAPL  183.050003
8  2024-05-13   AAPL  186.279999
9  2024-05-14   AAPL  187.429993
10 2024-05-15   AAPL  189.720001
11 2024-05-16   AAPL  189.839996
12 2024-05-17   AAPL  189.869995
13 2024-05-20   AAPL  191.039993
14 2024-05-21   AAPL  192.350006
15 2024-05-22   AAPL  190.899994
16 2024-05-23   AAPL  186.880005
17 2024-05-24   AAPL  189.979996
18 2024-05-28   AAPL  189.990005
19 2024-05-29   AAPL  190.289993

Fundamental Data:
  ticker quarter  earnings        revenue
0   AAPL     N/A      6.44  381623.009280
1   MSFT     N/A     11.56  236583.993344
2  GOOGL     N/A      6.52  318145.986560
3   AMZN     N/A      3.57  590739.996672
4   META     N/A     17.36  142711.996416
5   TSLA     

In [3]:
historical_price_data

,date,ticker,close
0,2024-05-01,AAPL,169.300003
1,2024-05-02,AAPL,173.029999
2,2024-05-03,AAPL,183.380005
3,2024-05-06,AAPL,181.710007
4,2024-05-07,AAPL,182.399994
...,...,...,...
199,2024-06-12,TSLA,177.289993
200,2024-06-13,TSLA,182.470001
201,2024-06-14,TSLA,178.009995
202,2024-06-17,TSLA,187.440002


In [4]:
fundamental_data

,ticker,quarter,earnings,revenue
0,AAPL,N/A,6.44,381623.009280
1,MSFT,N/A,11.56,236583.993344
2,GOOGL,N/A,6.52,318145.986560
3,AMZN,N/A,3.57,590739.996672
4,META,N/A,17.36,142711.996416
5,TSLA,N/A,3.92,94745.001984


In [5]:
trade_orders

,order_id,date,ticker,quantity,price
0,1,2024-06-01,AAPL,10,150
1,2,2024-06-01,MSFT,5,200
2,3,2024-06-02,AAPL,15,155
3,4,2024-06-02,GOOGL,7,2850
4,5,2024-06-03,AMZN,20,3200
5,6,2024-06-03,META,8,250
6,7,2024-06-04,TSLA,12,700
7,8,2024-06-04,AAPL,10,152
8,9,2024-06-05,MSFT,6,202
9,10,2024-06-05,GOOGL,9,2880


In [6]:
strategy_signals

,date,ticker,signal
0,2024-06-01,AAPL,buy
1,2024-06-01,MSFT,sell
2,2024-06-02,AAPL,buy
3,2024-06-02,GOOGL,sell
4,2024-06-03,AMZN,buy
5,2024-06-03,META,sell
6,2024-06-04,TSLA,buy
7,2024-06-04,AAPL,sell
8,2024-06-05,MSFT,buy
9,2024-06-05,GOOGL,sell


In [7]:
volume_data

,date,ticker,volume
0,2024-05-01,AAPL,50383100
1,2024-05-02,AAPL,94214900
2,2024-05-03,AAPL,163224100
3,2024-05-06,AAPL,78569700
4,2024-05-07,AAPL,77305800
...,...,...,...
199,2024-06-12,TSLA,90389400
200,2024-06-13,TSLA,118984100
201,2024-06-14,TSLA,82038200
202,2024-06-17,TSLA,109786100


In [8]:
# Join
# pd.concat
# Ignore_index
# Multi-index -> fetch using iloc
# Concat dataframes horizontally



In [9]:
historical_price_data
fundamental_data
volume_data
trade_orders
strategy_signals

,date,ticker,signal
0,2024-06-01,AAPL,buy
1,2024-06-01,MSFT,sell
2,2024-06-02,AAPL,buy
3,2024-06-02,GOOGL,sell
4,2024-06-03,AMZN,buy
5,2024-06-03,META,sell
6,2024-06-04,TSLA,buy
7,2024-06-04,AAPL,sell
8,2024-06-05,MSFT,buy
9,2024-06-05,GOOGL,sell


In [10]:
historical_price_data.set_index(['date','ticker']).join(fundamental_data.set_index('ticker'),how="outer")

,,close,quarter,earnings,revenue
date,ticker,,,,
2024-05-01,AAPL,169.300003,N/A,6.44,381623.009280
2024-05-02,AAPL,173.029999,N/A,6.44,381623.009280
2024-05-03,AAPL,183.380005,N/A,6.44,381623.009280
2024-05-06,AAPL,181.710007,N/A,6.44,381623.009280
2024-05-07,AAPL,182.399994,N/A,6.44,381623.009280
...,...,...,...,...,...
2024-06-12,TSLA,177.289993,N/A,3.92,94745.001984
2024-06-13,TSLA,182.470001,N/A,3.92,94745.001984
2024-06-14,TSLA,178.009995,N/A,3.92,94745.001984


In [11]:
historical_price_data

,date,ticker,close
0,2024-05-01,AAPL,169.300003
1,2024-05-02,AAPL,173.029999
2,2024-05-03,AAPL,183.380005
3,2024-05-06,AAPL,181.710007
4,2024-05-07,AAPL,182.399994
...,...,...,...
199,2024-06-12,TSLA,177.289993
200,2024-06-13,TSLA,182.470001
201,2024-06-14,TSLA,178.009995
202,2024-06-17,TSLA,187.440002


In [12]:
fundamental_data

,ticker,quarter,earnings,revenue
0,AAPL,N/A,6.44,381623.009280
1,MSFT,N/A,11.56,236583.993344
2,GOOGL,N/A,6.52,318145.986560
3,AMZN,N/A,3.57,590739.996672
4,META,N/A,17.36,142711.996416
5,TSLA,N/A,3.92,94745.001984


In [13]:
historical_price_data.shape

(204, 3)

In [14]:
fundamental_data.shape

(6, 4)

In [15]:
pd.concat([historical_price_data,fundamental_data],axis=1)

,date,ticker,close,ticker,quarter,earnings,revenue
0,2024-05-01,AAPL,169.300003,AAPL,N/A,6.44,381623.009280
1,2024-05-02,AAPL,173.029999,MSFT,N/A,11.56,236583.993344
2,2024-05-03,AAPL,183.380005,GOOGL,N/A,6.52,318145.986560
3,2024-05-06,AAPL,181.710007,AMZN,N/A,3.57,590739.996672
4,2024-05-07,AAPL,182.399994,META,N/A,17.36,142711.996416
...,...,...,...,...,...,...,...
199,2024-06-12,TSLA,177.289993,NaN,NaN,NaN,NaN
200,2024-06-13,TSLA,182.470001,NaN,NaN,NaN,NaN
201,2024-06-14,TSLA,178.009995,NaN,NaN,NaN,NaN
202,2024-06-17,TSLA,187.440002,NaN,NaN,NaN,NaN


In [16]:
pd.concat([historical_price_data,fundamental_data],ignore_index=True)

,date,ticker,close,quarter,earnings,revenue
0,2024-05-01,AAPL,169.300003,NaN,NaN,NaN
1,2024-05-02,AAPL,173.029999,NaN,NaN,NaN
2,2024-05-03,AAPL,183.380005,NaN,NaN,NaN
3,2024-05-06,AAPL,181.710007,NaN,NaN,NaN
4,2024-05-07,AAPL,182.399994,NaN,NaN,NaN
...,...,...,...,...,...,...
205,NaT,MSFT,NaN,N/A,11.56,236583.993344
206,NaT,GOOGL,NaN,N/A,6.52,318145.986560
207,NaT,AMZN,NaN,N/A,3.57,590739.996672
208,NaT,META,NaN,N/A,17.36,142711.996416


In [17]:
pd.concat([historical_price_data,fundamental_data],keys=["Hist Data","Fund Data"])
# Multi Index

date ticker       close quarter  earnings        revenue
Hist Data 0 2024-05-01   AAPL  169.300003     NaN       NaN            NaN
          1 2024-05-02   AAPL  173.029999     NaN       NaN            NaN
          2 2024-05-03   AAPL  183.380005     NaN       NaN            NaN
          3 2024-05-06   AAPL  181.710007     NaN       NaN            NaN
          4 2024-05-07   AAPL  182.399994     NaN       NaN            NaN
...                ...    ...         ...     ...       ...            ...
Fund Data 1        NaT   MSFT         NaN     N/A     11.56  236583.993344
          2        NaT  GOOGL         NaN     N/A      6.52  318145.986560
          3        NaT   AMZN         NaN     N/A      3.57  590739.996672
          4        NaT   META         NaN     N/A     17.36  142711.996416
          5        NaT   TSLA         NaN     N/A      3.92   94745.001984

[210 rows x 6 columns]

In [107]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define two separate lists of tickers for two datasets
tickers_set1 = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
tickers_set2 = ['MSFT', 'META', 'TSLA', 'AAPL']

# Fetch historical price data for the first set of tickers
def fetch_historical_price_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    if 'Close' in data:
        data = data['Close'].reset_index().melt(id_vars=['Date'], var_name='Ticker', value_name='Close')
        data.columns = ['Date', 'Ticker', 'Close']
    else:
        data = pd.DataFrame(columns=['Date', 'Ticker', 'Close'])
    return data

# Fetch fundamental data for the second set of tickers
def fetch_fundamental_data(tickers):
    fundamentals = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info
        revenue = info.get('totalRevenue')
        fundamentals.append({
            'Ticker': ticker,
            'Quarter': 'N/A',  # Quarterly data is not directly available
            'Earnings': info.get('trailingEps'),
            'Revenue': revenue / 1e6 if revenue else None,  # Convert to millions, handle None
            'P/E Ratio': info.get('trailingPE')  # Additional common column
        })
    return pd.DataFrame(fundamentals)

# Fetch data
start_date = '2024-05-01'
end_date = '2024-06-20'

historical_price_data = fetch_historical_price_data(tickers_set1, start_date, end_date)
fundamental_data = fetch_fundamental_data(tickers_set2)

# Add some common columns to historical_price_data for demonstration
earnings_values = [5.5, 10.2, 8.3, 6.5]
revenue_values = [110, 200, 180, 150]
historical_price_data['Earnings'] = np.tile(earnings_values, len(historical_price_data) // len(earnings_values) + 1)[:len(historical_price_data)]
historical_price_data['Revenue'] = np.tile(revenue_values, len(historical_price_data) // len(revenue_values) + 1)[:len(historical_price_data)]

# Ensure 'Earnings' and 'Revenue' are of the same type (float) in both DataFrames
historical_price_data['Earnings'] = historical_price_data['Earnings'].astype(float)
historical_price_data['Revenue'] = historical_price_data['Revenue'].astype(float)
fundamental_data['Earnings'] = fundamental_data['Earnings'].astype(float)
fundamental_data['Revenue'] = fundamental_data['Revenue'].astype(float)

# Display data
print("Historical Price Data:")
print(historical_price_data.head(20))

print("\nFundamental Data:")
print(fundamental_data)

# Example of merging data with different join types

# Inner Merge
merged_data_inner = pd.merge(historical_price_data, fundamental_data, on=['Ticker', 'Earnings', 'Revenue'], how='inner', suffixes=('', '_fundamental'))
print("\nInner Merge Result:")
print(merged_data_inner.head(20))

# Outer Merge
merged_data_outer = pd.merge(historical_price_data, fundamental_data, on=['Ticker', 'Earnings', 'Revenue'], how='outer', suffixes=('', '_fundamental'))
print("\nOuter Merge Result:")
print(merged_data_outer.head(20))

# Left Merge
merged_data_left = pd.merge(historical_price_data, fundamental_data, on=['Ticker', 'Earnings', 'Revenue'], how='left', suffixes=('', '_fundamental'))
print("\nLeft Merge Result:")
print(merged_data_left.head(20))

# Right Merge
merged_data_right = pd.merge(historical_price_data, fundamental_data, on=['Ticker', 'Earnings', 'Revenue'], how='right', suffixes=('', '_fundamental'))
print("\nRight Merge Result:")
print(merged_data_right.head(20))


[*********************100%%**********************]  4 of 4 completed

Historical Price Data:
         Date Ticker       Close  Earnings  Revenue
0  2024-05-01   AAPL  169.300003       5.5    110.0
1  2024-05-02   AAPL  173.029999      10.2    200.0
2  2024-05-03   AAPL  183.380005       8.3    180.0
3  2024-05-06   AAPL  181.710007       6.5    150.0
4  2024-05-07   AAPL  182.399994       5.5    110.0
5  2024-05-08   AAPL  182.740005      10.2    200.0
6  2024-05-09   AAPL  184.570007       8.3    180.0
7  2024-05-10   AAPL  183.050003       6.5    150.0
8  2024-05-13   AAPL  186.279999       5.5    110.0
9  2024-05-14   AAPL  187.429993      10.2    200.0
10 2024-05-15   AAPL  189.720001       8.3    180.0
11 2024-05-16   AAPL  189.839996       6.5    150.0
12 2024-05-17   AAPL  189.869995       5.5    110.0
13 2024-05-20   AAPL  191.039993      10.2    200.0
14 2024-05-21   AAPL  192.350006       8.3    180.0
15 2024-05-22   AAPL  190.899994       6.5    150.0
16 2024-05-23   AAPL  186.880005       5.5    110.0
17 2024-05-24   AAPL  189.979996      10.

In [108]:
historical_price_data

,Date,Ticker,Close,Earnings,Revenue
0,2024-05-01,AAPL,169.300003,5.5,110.0
1,2024-05-02,AAPL,173.029999,10.2,200.0
2,2024-05-03,AAPL,183.380005,8.3,180.0
3,2024-05-06,AAPL,181.710007,6.5,150.0
4,2024-05-07,AAPL,182.399994,5.5,110.0
...,...,...,...,...,...
131,2024-06-12,MSFT,441.059998,6.5,150.0
132,2024-06-13,MSFT,441.579987,5.5,110.0
133,2024-06-14,MSFT,442.570007,10.2,200.0
134,2024-06-17,MSFT,448.369995,8.3,180.0


In [109]:
fundamental_data

,Ticker,Quarter,Earnings,Revenue,P/E Ratio
0,MSFT,N/A,11.56,236583.993344,38.908302
1,META,N/A,17.36,142711.996416,28.501150
2,TSLA,N/A,3.92,94745.001984,46.686222
3,AAPL,N/A,6.44,381623.009280,32.218945


In [111]:
historical_price_data.merge(fundamental_data,on='Ticker',how='inner',suffixes=("","_funda"))

,Date,Ticker,Close,Earnings,Revenue,Quarter,P/E Ratio


In [112]:
historical_price_data.merge(fundamental_data,on='Ticker',how='left',suffixes=("","_funda"))

,Date,Ticker,Close,Earnings,Revenue,Quarter,P/E Ratio


In [99]:
historical_price_data.merge(fundamental_data,on='Ticker',how='right',suffixes=("","_funda"))

,Date,Ticker,Close,Earnings,Revenue,Quarter,Earnings_funda,Revenue_funda,P/E Ratio
0,2024-05-01,MSFT,394.940002,8.3,180.0,N/A,11.56,236583.993344,38.908302
1,2024-05-02,MSFT,397.839996,6.5,150.0,N/A,11.56,236583.993344,38.908302
2,2024-05-03,MSFT,406.660004,5.5,110.0,N/A,11.56,236583.993344,38.908302
3,2024-05-06,MSFT,413.540009,10.2,200.0,N/A,11.56,236583.993344,38.908302
4,2024-05-07,MSFT,409.339996,8.3,180.0,N/A,11.56,236583.993344,38.908302
...,...,...,...,...,...,...,...,...,...
65,2024-06-12,AAPL,213.070007,10.2,200.0,N/A,6.44,381623.009280,32.218945
66,2024-06-13,AAPL,214.240005,8.3,180.0,N/A,6.44,381623.009280,32.218945
67,2024-06-14,AAPL,212.490005,6.5,150.0,N/A,6.44,381623.009280,32.218945
68,2024-06-17,AAPL,216.669998,5.5,110.0,N/A,6.44,381623.009280,32.218945


In [100]:
historical_price_data.merge(fundamental_data,on='Ticker',how='outer',suffixes=("","_funda"))

,Date,Ticker,Close,Earnings,Revenue,Quarter,Earnings_funda,Revenue_funda,P/E Ratio
0,2024-05-01,AAPL,169.300003,5.5,110.0,N/A,6.44,381623.009280,32.218945
1,2024-05-02,AAPL,173.029999,10.2,200.0,N/A,6.44,381623.009280,32.218945
2,2024-05-03,AAPL,183.380005,8.3,180.0,N/A,6.44,381623.009280,32.218945
3,2024-05-06,AAPL,181.710007,6.5,150.0,N/A,6.44,381623.009280,32.218945
4,2024-05-07,AAPL,182.399994,5.5,110.0,N/A,6.44,381623.009280,32.218945
...,...,...,...,...,...,...,...,...,...
133,2024-06-13,MSFT,441.579987,5.5,110.0,N/A,11.56,236583.993344,38.908302
134,2024-06-14,MSFT,442.570007,10.2,200.0,N/A,11.56,236583.993344,38.908302
135,2024-06-17,MSFT,448.369995,8.3,180.0,N/A,11.56,236583.993344,38.908302
136,2024-06-18,MSFT,446.339996,6.5,150.0,N/A,11.56,236583.993344,38.908302


In [115]:
pd.merge(historical_price_data, fundamental_data)


,Date,Ticker,Close,Earnings,Revenue,Quarter,P/E Ratio


In [116]:
historical_price_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      136 non-null    datetime64[ns]
 1   Ticker    136 non-null    object        
 2   Close     136 non-null    float64       
 3   Earnings  136 non-null    float64       
 4   Revenue   136 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 5.4+ KB


In [117]:
fundamental_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Ticker     4 non-null      object 
 1   Quarter    4 non-null      object 
 2   Earnings   4 non-null      float64
 3   Revenue    4 non-null      float64
 4   P/E Ratio  4 non-null      float64
dtypes: float64(3), object(2)
memory usage: 292.0+ bytes
